In [1]:
import wikipedia
import pandas as pd
import spacy
import random
import torch
import numpy as np
import transformers 
import os
import re
import argparse as ap
import time
import nltk

from sentence_transformers import SentenceTransformer
from FlagEmbedding import BGEM3FlagModel
from openai import OpenAI
from scipy.stats import spearmanr
from transformers import pipeline
from huggingface_hub import login

In [13]:
def noun_list(a, lang):
    """
    Filtra la pregunta y obtiene las PoST relevantes.
    
    a = list; Lista de preguntas del dataset
    lang = 'es' or 'en'; Idioma a trabajar
    """
    if lang == 'es':
        post_spacy = spacy.load("es_core_news_sm")
    else:
        post_spacy = spacy.load("en_core_web_sm")
    noun_list = []
    nums = ['0','1','2','3','4','5','6','7','8','9']

    for _ in a:
        doc = post_spacy(_)
        sub_noun = []
        for token in doc:
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "NUM":
                sub_noun.append(token.text)
            if token.pos_ == "ADJ" and token.text[0] in nums:
                sub_noun.append(token.text)
        if not sub_noun: 
            noun_list.append(_) 
        else: 
            noun_list.append(sub_noun) 
    return noun_list


# FUNCIONES ADRIÁAAAAAAAAAAAAAAN
# -------------------------------------------------
def sent_noun_list(b, lang):
    """"
    Devuelve una lista con sublistas de PoST por cada oración en la respuesta.

     b = list; Lista de respuestas del dataset
    lang = 'es' or 'en'; Idioma a trabajar
    """
    if lang == 'es':
         post_spacy = spacy.load("es_core_news_sm")
    else:
        post_spacy = spacy.load("en_core_web_sm")
    sent_noun_list = []
    nums = ['0','1','2','3','4','5','6','7','8','9']

    for _ in b:
        doc = post_spacy(_)
        sent_token_list = []
        for sent in doc.sents:
            token_list = []
            for token in sent:
                if token.pos_ == "PROPN" or  token.pos_ == "NOUN" or token.pos_ == "NUM":
                    token_list.append(token.text)
            sent_token_list.append(token_list)
        sent_noun_list.append(sent_token_list)
    return sent_noun_list

def one_sent(n_list, answers_list):
    """
    Devuelve una lista con los sustantivos de la oración que (en teoría) responde a cada pregunta.
    n_list = list; Obtenida de la función noun_list().
    answers_list = list; Obtenida de la función noun_answer().
    """
    one_sentence_list = []
    for i, sublist in enumerate(n_list):
        j_0 = 0
        best_match = None
        for other_sublist in answers_list[i]:
            sublist_set = set(sublist)
            other_sublist_set = set(other_sublist)            
            common_elements = sublist_set & other_sublist_set
            count = len(common_elements)
            if count >= j_0:
                j_0 = count
                best_match = other_sublist  
        one_sentence_list.append(best_match)  
    return one_sentence_list


def complement(n_list,one_sent_list):
    """"
    Devuelve la lista obtenida de la resta de n_list con one_sentence_list. Si la oración tiene exactamente los mismos términos, devuelve estos.
    n_list = list; Obtenida de la función noun_list().
    one_sentence_list =  list; Obtenida de la función one_sent
    """
    result_list = []
    for i,sublist in enumerate(n_list):
        diff_list = list(set(sublist)-set(one_sent_list[i]))
        if diff_list == []:
            diff_list = sublist
        result_list.append(diff_list)
    return result_list


# -------------------------------------------------------------------------
def keyword_por_preg(n_list):
    """
    Junta lista de PoST previo a pasarlo por el API de Wikipedia.
    
    n_list = list; Obtenida de la función noun_list().
    """
    keyword_list = []
    for i in n_list:
        keyword = ''
        for j in i:
            keyword = keyword + j + ' '
        keyword_list.append(keyword)
    return keyword_list


# FUNCIONES ADRIÁAAAAAAAAAAAAAAAAAAAN
# ---------------------------------------------------
def list_to_dict(a_en):
    """
    Convierte la lista de preguntas en un diccionario con llave la pregunta, y valores, los índices donde se repite.
    a_en = list; df de preguntas
    """
    dict_preguntas = {}
    for i, preg in enumerate(a_en):
        if preg not in dict_preguntas:
            dict_preguntas[preg] = set()
        dict_preguntas[preg].add(i)
    return dict_preguntas

def comprimir(dict_preguntas,one_sentence_list):
    """
    Para cada pregunta elige el conjunto de tokens de la respuesta del modelo que más contexto aporte (en cantidad de palabras). Devuelve una lista
    para pasarle a wikipipeline()
    dict_preguntas = dict; se obtiene de list_to_dict()
    one_sentence_list = list; se obtiene de one_sent()
    """
    short_tokens_list = []
    for i in dict_preguntas:
        largest_context = 0
        for j in dict_preguntas[i]:
            if len(one_sentence_list[int(j)]) >= largest_context:
                largest_context = int(j)
        short_tokens_list.append(one_sentence_list[largest_context])
    return short_tokens_list
# ---------------------------------------------------



def get_wikipage(text, lang, page_total):
    """
    Regresa las n páginas de Wikipedia más relevantes al query

    text = str; Texto proveniente de la función keyword_por_preg()
    lang = 'es' or 'en'; Lenguaje necesario para wikipedia
    page_total = int; Cantidad de páginas a regresar
    """
    if lang == 'es':
        wikipedia.set_lang('es')
    if lang == 'en':
        wikipedia.set_lang('en')
    page_title = wikipedia.search(text, results = page_total)
    return page_title


def wikipipeline(dataset, lang, page_count):
    """
    Genera los resúmenes que sirven como contexto de cada pregunta. 

    dataset = pd.DataFrame ; El nombre del dataset a procesar
    dataset = list; ya sea el nombre del dataset en formato dataset["model_input"] o list(set(dataset["model_input"]))
    lang = 'es' or 'en'; Idioma a trabajar, debe de coincidir con el del dataset para no generar algo incoherente
    page_count = int; Cantidad de páginas de Wikipedia a extraer
    """
    
    noun_list_perrona = noun_list(dataset, lang)
    key_list = keyword_por_preg(noun_list_perrona)

    resumen_list = []
    question_list = []
    iterador = 0
    for i in key_list:
        pages = get_wikipage(i, lang, page_count)
        resumen = ''
        for x in pages:
            try:
                page = wikipedia.WikipediaPage(x)
                page_sum = page.summary
                resumen = resumen + '' + page_sum
            except wikipedia.exceptions.DisambiguationError: # Se usa para evitar problemas al encontrar la página adecuada.
                print(i, x)
        resumen_list.append(resumen)
        question_list.append(dataset[iterador])
        iterador += 1
    return resumen_list, question_list


def generate_embeddings(sum_list, q, ruta):
    """
    Genera dataframe de embeddings, y los guarda en un directorio de nuestra elección.

    sum_list = list ; Lista de resúmenes obtenida previamente.
    q = list ; Lista de preguntas obtenida previamente.
    ruta = str ; Directorio para guardar. 
    """
    len_list = [len(_) for _ in sum_list]
    max_length = max(len_list)

    embs = model.encode(
        sum_list,
        batch_size = 12,
        max_length = max_length,
    )['dense_vecs']

    embs_loco = [_ for _ in embs]
    dic = {'Embedding':embs_loco, 'Texto':sum_list, 'Keywords Pregunta': q}
    embs_df = pd.DataFrame(data=dic)
    embs_df.to_csv(ruta)
    print(f"Embedding guardados en la ruta {ruta} . Saludos")


def full_context_pipeline(dataset, lang, num, ruta):
    """
    Ejecuta todo el pipeline (todo junto alv compa).
    
    dataset = pd.DataFrame[_column_name_] ;  Columna del dataset a trabajar
    lang = 'en' or 'es'; Lenguaje a trabajar.
    num = int; Cantidad de páginas de wikipedia a recolectar.
    ruta = str; Dirección de guardado de dataset. (DEBE DE TENER NOMBRE DEL ARCHIVO)
    """
    conjunto = list(set(dataset))
    sum_set, q_set = wikipipeline(conjunto, lang, num)
    generate_embeddings(sum_set, q_set, ruta)

In [4]:
df_es = pd.read_json(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\test_ds\v1\mushroom.es-tst.v1.jsonl', lines=True)
df_es

,id,lang,model_input,model_output_text,model_id,model_output_tokens,model_output_logits
0,tst-es-1,ES,¿En qué equipos de la NHL jugó Bert Olmstead d...,Bert Olmstead jugó para los Montreal Canadiens...,Iker/Llama-3-Instruct-Neurona-8b-v2,"[B, ert, ĠOl, m, stead, Ġjug, Ã³, Ġpara, Ġlos,...","[26.1876564026, 30.4659004211, 27.9287395477, ..."
1,tst-es-2,ES,"¿Cuántos habitantes tiene Forstescue, Misuri?","Según el Bureau del Censo de Estados Unidos, l...",Iker/Llama-3-Instruct-Neurona-8b-v2,"[Seg, Ãºn, Ġel, ĠBureau, Ġdel, ĠC, enso, Ġde, ...","[21.147600174, 28.2901725769, 20.9695224762, 1..."
2,tst-es-3,ES,¿En dónde se han encontrado restos de la balle...,Los restos fósiles de la ballena picuda de Bah...,Qwen/Qwen2-7B-Instruct,"[Los, Ġrest, os, Ġf, Ã³s, iles, Ġde, Ġla, Ġbal...","[20.4515380859, 20.4586791992, 31.832113266, 1..."
3,tst-es-4,ES,¿Quién ganó el premio BAFTA al mejor actor de ...,"Según la información disponible, el premio BAF...",meta-llama/Meta-Llama-3-8B-Instruct,"[Seg, Ãºn, Ġla, ĠinformaciÃ³n, Ġdisponible, ,,...","[20.6939048767, 28.1913757324, 23.7558517456, ..."
4,tst-es-5,ES,¿Cuántas medidas disciplinares se aprobaron en...,"El Segundo Concilio de Nicea, también conocido...",Iker/Llama-3-Instruct-Neurona-8b-v2,"[El, ĠSeg, undo, ĠConc, ilio, Ġde, ĠNice, a, ,...","[19.8374900818, 21.1202754974, 34.8290023804, ..."
...,...,...,...,...,...,...,...
147,tst-es-148,ES,¿Cuántas veces ha sido seleccionado Domantas S...,Hasta la fecha de mi última actualización en o...,Qwen/Qwen2-7B-Instruct,"[H, asta, Ġla, Ġfecha, Ġde, Ġmi, ĠÃºltima, Ġac...","[20.6600112915, 27.0722904205, 18.8246574402, ..."
148,tst-es-149,ES,¿Qué líneas de metro puedo tomar desde la esta...,"Desde la estación de République en París, pued...",Iker/Llama-3-Instruct-Neurona-8b-v2,"[Desde, Ġla, Ġest, aciÃ³n, Ġde, ĠRÃ©, pub, liq...","[21.195646286, 24.2160949707, 23.9696769714, 3..."
149,tst-es-150,ES,¿A qué altura se localiza el municipio de Yush...,"El municipio de Yushu, ubicado en la provincia...",Qwen/Qwen2-7B-Instruct,"[El, Ġmunicip, io, Ġde, ĠY, ush, u, ,, Ġubic, ...","[27.9522533417, 21.2112312317, 33.8999862671, ..."
150,tst-es-151,ES,¿En qué años se produjo la primera generación ...,La primera generación del Aston Martin Vanquis...,Qwen/Qwen2-7B-Instruct,"[La, Ġprimera, Ġgener, aciÃ³n, Ġdel, ĠAston, Ġ...","[27.9550552368, 24.7381954193, 25.8791179657, ..."


In [7]:
df_input = [i for i in df_es["model_input"]] # a
df_output = [i for i in df_es["model_output_text"]] # b

In [82]:
%%time
n_list = noun_list(df_input, 'es')
sent_list = sent_noun_list(df_output, 'es')
one_sent_list = one_sent(n_list, sent_list)
final_list = complement(n_list, one_sent_list)

# Corrección de final_list
for i in final_list:
    if len(i[0]) < 2:
        index = final_list.index(i)
        aux = []
        for _ in n_list[index].split():
            aux.append(_)
        final_list[index] = aux

keyword_list = keyword_por_preg(final_list)
dict_preg = list_to_dict(df_input)
short_n_list = list(dict_preg.keys())
short_tokens_list = comprimir(dict_preg,keyword_list)
print(len(short_tokens_list))
print(short_tokens_list)

152
['equipos ', 'Forstescue ', 'restos ballena Bahamonde Mesoplodon ', 'premio BAFTA actor reparto año 2000 película ', 'consejo medidas ', 'copas mundo futbolista ', 'colina ', 'años ', 'película Sophie Nélisse papel Liesel Meminger ', 'vocalista grupo Mamamoo ', 'Argenitna copa equipo año mundo ', 'John Swinney Partido Nacional Escocés ', 'inundaciones zonas marisma Inglaterra Fens Fenland ', 'nombre ', 'república ', 'animal ', 'causas diabetes tipo 2 ', 'guerras ', 'industrias Folorunsho Alakija ', 'cosmología ', 'deporte ', 'ministra Togo 2020 ', 'ciudades Cúantas ', 'norte dos Rodesias países sur ', 'lugar ', 'Alóadas mitología ', '¿Qué es un retrónimo? ', 'país ', 'fase formación montañas Alpes ', 'tres protagonistas película Faster Pussycat Kill ', 'planta tipo ', 'causas ', 'país ', 'Secretario Marina Estados Unidos 2017 2019 ', 'año ', 'medallas selección voleibol Cuba ', 'disco The Eternal Idol Black Sabbath ', 'origen nombre ', 'residencia presidente Colombia ', 'director p

In [81]:
print(df_input[0], "|", df_output[0])
print("-------")
print(n_list[0], sent_list[0])
print("-------")
print(one_sent_list[0])
print("-------")
print(final_list[0])

¿En qué equipos de la NHL jugó Bert Olmstead durante su carrera? | Bert Olmstead jugó para los Montreal Canadiens, Toronto Maple Leafs, New York Rangers y Los Angeles Kings durante su carrera en la NHL.
-------
['equipos', 'NHL', 'Bert', 'Olmstead', 'carrera'] [['Bert', 'Olmstead', 'Montreal', 'Canadiens', 'Toronto', 'Maple', 'Leafs', 'New', 'York', 'Rangers', 'Angeles', 'Kings', 'carrera', 'NHL']]
-------
['Bert', 'Olmstead', 'Montreal', 'Canadiens', 'Toronto', 'Maple', 'Leafs', 'New', 'York', 'Rangers', 'Angeles', 'Kings', 'carrera', 'NHL']
-------
['equipos']


In [62]:
noun_list_perrona = noun_list(df_input, 'es')
key_list = keyword_por_preg(noun_list_perrona)
key_list

['equipos NHL Bert Olmstead carrera ',
 'habitantes Forstescue Misuri ',
 'restos ballena Bahamonde Mesoplodon ',
 'premio BAFTA actor reparto año 2000 película ',
 'medidas consejo Nicea año 787 ',
 'copas mundo futbolista Mario Bolatti ',
 'colina Lituania ',
 'años provincia Korçë Albania ',
 'película Sophie Nélisse papel Liesel Meminger ',
 'vocalista grupo Mamamoo ',
 'año Argenitna copa mundo Lionel Messi capitán equipo ',
 'John Swinney Partido Nacional Escocés ',
 'inundaciones zonas marisma Inglaterra Fens Fenland ',
 'nombre licencias Microsoft código fuente ',
 'república Rusia ciudad Kaspisk ',
 'tipo animal Hylarana ',
 'causas diabetes tipo 2 ',
 'guerras Adolf Galland ',
 'industrias Folorunsho Alakija ',
 'cosmología ',
 'deporte atleta Gergely Kulcsar ',
 'ministra Togo 2020 ',
 'Cúantas ciudades condado Gogebic Michigan ',
 'países dos Rodesias Rodesia norte sur ',
 'lugar John Christopher Willis ',
 'Alóadas mitología ',
 '¿ Q u é   e s   u n   r e t r ó n i m o ? '

In [84]:
for i in range(len(key_list)):
    print(key_list[i], "|", one_sent_list[i])
    print("-------")

equipos NHL Bert Olmstead carrera  | ['Bert', 'Olmstead', 'Montreal', 'Canadiens', 'Toronto', 'Maple', 'Leafs', 'New', 'York', 'Rangers', 'Angeles', 'Kings', 'carrera', 'NHL']
-------
habitantes Forstescue Misuri  | ['Bureau', 'Censo', 'Estados', 'Unidos', 'población', 'Foristec', 'Misuri', 'partir', 'año', '2019', '8.590', 'habitantes']
-------
restos ballena Bahamonde Mesoplodon  | ['restos', 'ballena', 'Bahamonde', 'Mesoplodon', 'traversii', 'América', 'Norte', 'estados', 'California', 'Oregon']
-------
premio BAFTA actor reparto año 2000 película  | ['información', 'premio', 'BAFTA', 'actor', 'reparto', 'año', '2000', 'Michael', 'Caine', 'papel', 'película', 'The', 'Cider', 'House', 'Rules', '1999']
-------
medidas consejo Nicea año 787  | ['Concilio', 'Nicea', 'Concilio', 'Constantinopla', 'año', '787', 'd.C.', 'concilio', 'objetivo', 'cuestión', 'iconoclasia', 'debate', 'adoración', 'imágenes']
-------
copas mundo futbolista Mario Bolatti  | ['Mario', 'Bolatti', 'tres', 'Copas', 

In [76]:
for i in range(len(key_list)):
    aux = key_list[i].split()
    aux2 = short_tokens_list[i].split()
    if set(aux) != set(aux2):
        for _ in aux2:
            if _ in aux:
                aux.remove(_)
    print(key_list[i], "|", aux)

equipos NHL Bert Olmstead carrera  | ['NHL', 'Bert', 'Olmstead', 'carrera']
habitantes Forstescue Misuri  | ['habitantes', 'Misuri']
restos ballena Bahamonde Mesoplodon  | ['restos', 'ballena', 'Bahamonde', 'Mesoplodon']
premio BAFTA actor reparto año 2000 película  | ['premio', 'BAFTA', 'actor', 'reparto', 'año', '2000', 'película']
medidas consejo Nicea año 787  | ['Nicea', 'año', '787']
copas mundo futbolista Mario Bolatti  | ['Mario', 'Bolatti']
colina Lituania  | ['Lituania']
años provincia Korçë Albania  | ['provincia', 'Korçë', 'Albania']
película Sophie Nélisse papel Liesel Meminger  | ['película', 'Sophie', 'Nélisse', 'papel', 'Liesel', 'Meminger']
vocalista grupo Mamamoo  | ['vocalista', 'grupo', 'Mamamoo']
año Argenitna copa mundo Lionel Messi capitán equipo  | ['Lionel', 'Messi', 'capitán']
John Swinney Partido Nacional Escocés  | ['John', 'Swinney', 'Partido', 'Nacional', 'Escocés']
inundaciones zonas marisma Inglaterra Fens Fenland  | ['inundaciones', 'zonas', 'marisma', 